In [1]:
import pandas as pd
import pymysql

In [8]:
def get_allowed_snp_names():
    df = pd.read_csv('allowed_snp.csv')
    df['snp'] = df['snp'].apply(lambda x : x.split('_')[1])
    df = df.sort_values(by=['snp'])
    snps = df['snp'].tolist()
    
    return snps

In [3]:
def preprocess_snp_data(file, sheet_name, target_file):
    df = pd.read_excel(file, sheet_name = sheet_name)
    allowed_snps = get_allowed_snp_names()
    columns = df.columns.tolist()
    final_columns = []
    for col in columns:
        if col.startswith('rs'):
            if col in allowed_snps:
                final_columns.append(col)
        elif col != 'qibie':
            final_columns.append(col)
    df = df[final_columns]
    df.to_csv(target_file, index=0)
        

In [4]:
def generator_train_and_valid_data():
    preprocess_snp_data('snp.xlsx', '1', '1.csv')
    preprocess_snp_data('snp.xlsx', '2', '2.csv')

In [10]:
generator_train_and_valid_data()

In [36]:
# 将excel数据转换为ped和info格式
def convert_to_ped(file):
    ped_df = pd.DataFrame()
    df = pd.read_csv(file)
    ped_df['family_id'] = df['Simple Name']
    ped_df['individual_id'] = df['Simple Name']
    ped_df['f_id'] = 0
    ped_df['m_id'] = 0
    ped_df['sex'] = df['sex']
    ped_df['case'] = df['caco'] + 1
    allowed_snps = get_allowed_snp_names()
    for snp in allowed_snps:
        snp_seri = df[snp]
        ped_df[snp+'_0']=snp_seri.apply(lambda x:x.split('/')[0])
        ped_df[snp+'_1'] = snp_seri.apply(lambda x:x.split('/')[1])
    return ped_df
    
    

In [38]:
ped_df = convert_to_ped('1.csv')
ped_df.to_csv('1.ped', index=0, sep='\t', header=0)

In [23]:
def get_snp_position():
    snps =  get_allowed_snp_names()
    con_str = ""
    for snp in snps:
        con_str += '"' + snp + '"' + ','
    con_str = con_str[:-1]
    db = pymysql.connect(host='genome-mysql.cse.ucsc.edu',
                     user='genome',
                     password='',
                     database='hg38')
    sql = 'select chrom,chromStart,chromEnd,name from snp151 where name in ({})'.format(con_str)
    cursor = db.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    snp_position_map = {}
    for row in result:
        chrom = row[0]
        start_position = row[1]
        end_position = row[2]
        name = row[3]
        snp_position_map[name] = start_position+1
        
    return snp_position_map
    
    

In [10]:
get_snp_position()

{'rs10091503': 20205843,
 'rs1026435': 138734165,
 'rs10503675': 20215320,
 'rs11204100': 20214096,
 'rs11681642': 70935956,
 'rs11778205': 20209107,
 'rs1200130': 169109334,
 'rs1200135': 169110030,
 'rs1200137': 169111197,
 'rs1358714': 169110181,
 'rs1537232': 38984222,
 'rs1678664': 38783036,
 'rs1678674': 38791603,
 'rs1678690': 38814090,
 'rs1678729': 38832433,
 'rs1738254': 38779994,
 'rs17412009': 20219086,
 'rs2061907': 38857553,
 'rs2070703': 160142070,
 'rs2072462': 70963254,
 'rs2266918': 70943677,
 'rs2289273': 10371917,
 'rs35678': 10338239,
 'rs3737094': 38990077,
 'rs3766031': 169115881,
 'rs3807154': 138732973,
 'rs3823430': 38783112,
 'rs4452640': 38907976,
 'rs4714192': 38883893,
 'rs6458080': 38875650,
 'rs751122': 10378403,
 'rs874808': 38805517,
 'rs919390': 41966898,
 'rs9357283': 38832388}

In [31]:

def generator_snp_info_file(target_file):
    snp_df = pd.read_csv('allowed_snp.csv')
    new_df = pd.DataFrame()
    snp_pos_dict = get_snp_position()
    new_df['snp'] = snp_df['snp'].apply(lambda x: x.split('_')[1])
    new_df = new_df.sort_values(by = ['snp'])
    new_df['position'] = new_df['snp'].apply(lambda x: snp_pos_dict[x])
    new_df.to_csv(target_file, index=0, header=0, sep='\t')
    

In [32]:
generator_snp_info_file('1.info')